In [55]:
import pandas as pd
import glob
from tqdm import tqdm
import requests
import time
import os
import shutil
import matplotlib.pyplot as plt
import sys
import platform
from scipy.stats import zscore

def load_manifest_file(location):
    """
    Function use:
    Load the GDC manifest file from a specified directory.

    Inputs:
    - location (str): Directory path containing the GDC manifest file.

    Outputs:
    - DataFrame: Contains the data from the manifest file if found.
    - None: If no matching file is found.
    """
    pattern = f"{location}/*gdc_manifest*"
    files = glob.glob(pattern)
    if files:
        return pd.read_csv(files[0], sep='\t')
    else:
        return None

def fetch_sample_type_from_file_uuid(file_uuid):
    """
    Function use:
    Fetch the sample type for a given file UUID using the GDC API.

    Inputs:
    - file_uuid (str): UUID of the file.

    Outputs:
    - sample_type (str): Sample type (e.g., 'Primary Tumor').
    - None: If the API request fails.
    """
    base_url = 'https://api.gdc.cancer.gov'
    response = requests.get(f'{base_url}/files/{file_uuid}?expand=cases.samples')
    if response.status_code == 200:
        data = response.json()
        return data.get('data', {}).get('cases', [{}])[0].get('samples', [{}])[0].get('sample_type', 'Not Found')
    else:
        print(f"Failed to retrieve sample type for UUID {file_uuid}.")
        return None

def fetch_diagnosis_details_from_file_uuid(file_uuid):
    """
    Function use:
    Fetch diagnosis details for a given file UUID using the GDC API.

    Inputs:
    - file_uuid (str): UUID of the file.

    Outputs:
    - diagnosis_details (list): List of dictionaries containing diagnosis information.
    - Empty list: If the API request fails.
    """
    base_url = 'https://api.gdc.cancer.gov'
    response = requests.get(f'{base_url}/files/{file_uuid}?expand=cases.diagnoses')
    if response.status_code == 200:
        data = response.json()
        return data.get('data', {}).get('cases', [{}])[0].get('diagnoses', [])
    else:
        print(f"Failed to retrieve diagnosis details for UUID {file_uuid}.")
        return []

def fetch_clinical_data_for_manifest(manifest, n=None, t=1):
    """
    Function use:
    Enrich a manifest DataFrame with sample types and diagnosis details fetched from the GDC API.

    Inputs:
    - manifest (DataFrame): Manifest with file UUIDs in an 'id' column.
    - n (int): Maximum number of files to process. If None, all rows are processed.
    - t (float): Time (in seconds) to wait between API calls.

    Outputs:
    - Updated DataFrame: Includes additional sample type and diagnosis details.
    """
    if 'sample_type' not in manifest.columns:
        manifest['sample_type'] = None

    if n is None:
        n = len(manifest)

    for i in tqdm(range(min(n, len(manifest)))):
        if i > 0:
            time.sleep(t)
        file_id = manifest.loc[i, 'id']
        sample_type = fetch_sample_type_from_file_uuid(file_id)
        if sample_type:
            manifest.at[i, 'sample_type'] = sample_type
        else:
            print(f"No sample type found for file UUID {file_id}.")
        diagnosis_details = fetch_diagnosis_details_from_file_uuid(file_id)
        if diagnosis_details and isinstance(diagnosis_details, list):
            for detail in diagnosis_details:
                if isinstance(detail, dict):
                    for key, value in detail.items():
                        column_name = f"diagnosis_{key}"
                        if column_name not in manifest.columns:
                            manifest[column_name] = None
                        manifest.at[i, column_name] = value
                else:
                    print(f"Detail is not a dictionary: {detail}")
        else:
            print(f"Diagnosis details structure unexpected or missing for file ID {file_id}: {diagnosis_details}")

    return manifest

def save_clinical_manifest(manifest, directory):
    """
    Function use:
    Save the clinical manifest DataFrame to a file.

    Inputs:
    - manifest (DataFrame): Data to save.
    - directory (str): Directory path where the file will be saved.

    Outputs:
    - None: File is saved directly to the specified directory.
    """
    directory = os.path.join(directory, '')
    filepath = f"{directory}clinical_gdc_manifest.txt"
    manifest.to_csv(filepath, sep='\t', index=False)
    print(f"File saved successfully at {filepath}")

def load_clinical_manifest(directory):
    """
    Function use:
    Load a clinical manifest file into a DataFrame.

    Inputs:
    - directory (str): Directory path where the file is located.

    Outputs:
    - DataFrame: Contains the clinical manifest data.
    """
    filepath = os.path.join(directory, 'clinical_gdc_manifest.txt')
    clinical_manifest = pd.read_csv(filepath, sep='\t')
    print('Clinical manifest shape:', clinical_manifest.shape)
    return clinical_manifest

def clean_dataframe(df):
    """
    Function use:
    Clean a DataFrame by removing columns with irrelevant or missing data.

    Inputs:
    - df (DataFrame): DataFrame to clean.

    Outputs:
    - DataFrame: Cleaned version of the input DataFrame.
    """
    df_copy = df.copy()
    for column in df_copy.columns:
        unique_values = df_copy[column].dropna().unique()
        if len(unique_values) == 0 or (len(unique_values) == 1 and unique_values[0] in ['Not Reported', 'not reported', 'None']):
            df_copy.drop(column, axis=1, inplace=True)
        else:
            if column in ["ajcc_pathologic_stage", "tumor_grade", "ajcc_pathologic_t", "ajcc_pathologic_m",
                          "ajcc_pathologic_n", "figo_stage", "wilms_tumor_histologic_subtype"]:
                df_copy.loc[df_copy['sample_type'] == 'Solid Tissue Normal', column] = 'healthy tissue'
    print('Cleaned clinical manifest shape:', df_copy.shape)
    return df_copy

def organize_tsv_files(base_dir):
    """
    Function use:
    Organize .tsv files by moving them from subdirectories to the base directory.

    Inputs:
    - base_dir (str): Path to the base directory.

    Outputs:
    - None: Files are moved, and subdirectories are cleaned.
    """
    has_directories = False
    for root, dirs, files in os.walk(base_dir, topdown=False):
        if dirs:
            has_directories = True
        for file in files:
            if file.endswith(".tsv"):
                shutil.move(os.path.join(root, file), os.path.join(base_dir, file))
    if has_directories:
        for root, dirs, files in os.walk(base_dir, topdown=False):
            for dir in dirs:
                shutil.rmtree(os.path.join(root, dir))
        print("Cleaning done")
    else:
        print("No directories to clean. Cleaning done.")

def load_gene_expression(manifest_df, exp_dir, feature):
    """
    Function use:
    Load gene expression data from .tsv files, keeping only relevant rows and columns.

    Inputs:
    - manifest_df (DataFrame): Manifest with filenames.
    - exp_dir (str): Directory containing .tsv files.
    - feature (str): Gene expression feature to extract.

    Outputs:
    - gene_expression_df (DataFrame): Contains gene expression data.
    - gene_key_df (DataFrame): Maps gene IDs to gene names.
    """
    gene_expression_data = {}
    gene_key = {}

    for filename in manifest_df['filename']:
        file_path = os.path.join(exp_dir, filename)
        try:
            df = pd.read_csv(file_path, sep='\t', comment='#')
            df = df[df['gene_id'].str.contains('ENSG')]
            gene_expression_data[filename] = df[feature].values
            gene_key.update(dict(zip(df['gene_id'], df['gene_name'])))
        except Exception as e:
            print(f"Failed to load {filename}: {e}")

    gene_expression_df = pd.DataFrame(gene_expression_data, index=list(gene_key.keys()))
    gene_expression_df.index.name = 'gene_id'
    gene_expression_df.columns.name = 'filename'

    gene_key_df = pd.DataFrame(list(gene_key.items()), columns=['gene_id', 'gene_name'])

    print('Gene expression shape:', gene_expression_df.T.shape)
    print('Gene key shape:', gene_key_df.shape)

    return gene_expression_df.T, gene_key_df

def save_dataframes(gene_exps, gene_key, cancer):
    """
    Function use:
    Save gene expression and key DataFrames to files.

    Inputs:
    - gene_exps (DataFrame): Gene expression data.
    - gene_key (DataFrame): Gene key data.
    - cancer (str): Directory for saving the data.

    Outputs:
    - None: DataFrames are saved to files.
    """
    gene_exps.to_csv(f'./data/TCGA_GeneExpression/{cancer}/gene_expression.csv', index=True)
    gene_key.to_csv(f'./data/TCGA_GeneExpression/{cancer}/gene_key.csv', index=True)

def load_dataframes(exps_filename, key_filename):
    """
    Function use:
    Load gene expression and key DataFrames from files.

    Inputs:
    - exps_filename (str): Path to gene expression file.
    - key_filename (str): Path to gene key file.

    Outputs:
    - gene_exps (DataFrame): Loaded gene expression data.
    - gene_key (DataFrame): Loaded gene key data.
    """
    gene_exps = pd.read_csv(exps_filename, index_col=0)
    gene_key = pd.read_csv(key_filename, index_col=0)
    return gene_exps, gene_key

def analyze_filenames(manifest_df, directory_path):
    """
    Analyze the uniqueness and overlap of filenames between a manifest and a directory.

    Inputs:
    - manifest_df (DataFrame): Manifest containing filenames.
    - directory_path (str): Directory path containing files to compare.

    Outputs:
    - dict: Summary of percentages including unique filenames, overlap, and non-overlap.
    """
    total_filenames = len(manifest_df['filename'])
    unique_filenames = len(manifest_df['filename'].unique())
    percent_unique = (unique_filenames / total_filenames) * 100

    directory_files = set(os.listdir(directory_path))
    manifest_files = set(manifest_df['filename'])

    overlap_files = manifest_files.intersection(directory_files)
    percent_overlap = (len(overlap_files) / total_filenames) * 100
    non_overlap_files = manifest_files.difference(directory_files)
    percent_non_overlap = (len(non_overlap_files) / total_filenames) * 100

    return {
        "percent_unique_filenames_in_manifest": percent_unique,
        "percent_overlap_with_directory": percent_overlap,
        "percent_non_overlap_with_directory": percent_non_overlap
    }

def annotate_clinical_manifest(clinical_manifest, debug_print = False):
    """
    Enhances the clinical manifest DataFrame with:
    - 'label': Indicates whether the sample is Cancer or Healthy based on 'sample_type'
    - 'cancer_stage': Simplified stage groupings
    - 'cancer_type': Based on diagnosis

    Prints the sample_type value counts and how they were classified.

    Returns:
    - Updated DataFrame with added 'label', 'cancer_type', and 'cancer_stage' columns.
    """

    df = clinical_manifest.copy()

    cancer_sample_types = [
        'Primary Tumor', 'Recurrent Tumor', 'Metastatic',
        'Next Generation Cancer Model', 'Expanded Next Generation Cancer Model',
        'Additional - New Primary'
    ]
    healthy_sample_types = ['Solid Tissue Normal', 'Bone Marrow Normal', 'Blood Derived Normal']

    for stype, count in df['sample_type'].value_counts().items():
        if stype in cancer_sample_types or 'Blood Derived Cancer' in stype:
            classification = 'Cancer'
        elif stype in healthy_sample_types:
            classification = 'Healthy'
        else:
            classification = 'Unknown'
        if debug_print:
            print(f"{stype:<60} ({count:>4}) → {classification}")

    df['label'] = df['sample_type'].apply(
        lambda x: 'Cancer' if x in cancer_sample_types or 'Blood Derived Cancer' in x
        else 'Healthy' if x in healthy_sample_types
        else 'Unknown'
    )

    stage_mapping = {
        'Stage IA': 'Stage I', 'Stage IA1': 'Stage I', 'Stage IA2': 'Stage I', 'Stage IA3': 'Stage I',
        'Stage IB': 'Stage I', 'Stage IC': 'Stage I', 
        'Stage IIA': 'Stage II', 'Stage IIB': 'Stage II', 'Stage IIC': 'Stage II',
        'Stage IIIA': 'Stage III', 'Stage IIIB': 'Stage III', 'Stage IIIC': 'Stage III',
        'Stage IIIC1': 'Stage II', 'Stage IIIC2': 'Stage II',
        'Stage IVA': 'Stage IV', 'Stage IVB': 'Stage IV', 'Stage IVC': 'Stage IV',
        'Stage X': 'Stage X',
        'Unknown': 'Unknown', 'Not Reported': 'Unknown'
    }

    df['cancer_stage'] = df['diagnosis_ajcc_pathologic_stage'].map(stage_mapping)
    df.loc[df['label'] == 'Healthy', 'cancer_stage'] = 'Healthy'
    df = df[df['label'] != 'Unknown']
    return df[['filename','OGT', 'OGA', 'label','cancer_stage']]

def add_multiple_gene_expressions_to_clinical(gene_exps, clinical_manifest, gene_key, gene_names):
    """
    Add multiple gene expressions to the clinical manifest DataFrame.

    Parameters:
    - gene_exps: DataFrame containing gene expressions.
    - clinical_manifest: DataFrame containing clinical manifest data.
    - gene_key: DataFrame containing gene IDs and names.
    - gene_names: List of gene names to be added.

    Returns:
    - Updated clinical manifest DataFrame with gene expressions.
    """
    for gene_name in gene_names:
        # Find the ENSG code for the given gene_name
        gene_id = gene_key.loc[gene_key['gene_name'] == gene_name, 'gene_id'].iloc[0]
        
        # Extract the relevant gene expression column
        gene_exps_reset = gene_exps.reset_index()
        gene_expression_series = gene_exps_reset.set_index('filename')[gene_id]
        
        # Add the gene expression data to the clinical manifest
        clinical_manifest[gene_name] = clinical_manifest['filename'].apply(lambda x: gene_expression_series.get(x, None))
    
    # Reordering columns to have gene names as the first columns
    cols = gene_names + [col for col in clinical_manifest if col not in gene_names]
    updated_clinical_manifest = clinical_manifest[cols]
    return updated_clinical_manifest

def load_process_tcga(cancer_name):
    """
    Processes TCGA cancer data unless a preprocessed CSV already exists.
    
    Saves cleaned expression data to: ./data/TCGA_GeneExpression/{cancer_name}/clean_expression_data.csv

    Returns:
    - DataFrame of cleaned and labeled clinical manifest, or None if file already exists.
    """
    base_path = f'./data/TCGA_GeneExpression/{cancer_name}/'
    output_path = os.path.join(base_path, 'clean_expression_data.csv')

    if os.path.exists(output_path):
        print(f"loading data: {output_path}")
        return pd.read_csv(output_path)

    print(f"[proccessing] data for: {cancer_name}")

    gene_names = ["OGT", "OGA"]

    clinical_manifest = load_clinical_manifest(base_path)
    gene_exps, gene_key = load_dataframes(
        os.path.join(base_path, 'gene_expression.csv'),
        os.path.join(base_path, 'gene_key.csv')
    )

    gene_exps = gene_exps.apply(zscore, axis=1)

    clinical_manifest_duo = add_multiple_gene_expressions_to_clinical(
        gene_exps, clinical_manifest, gene_key, gene_names
    )
    clinical_manifest_duo = annotate_clinical_manifest(clinical_manifest_duo)

    clinical_manifest_duo.to_csv(output_path, index=False)
    print(f"Cleaned data saved to: {output_path}\n")

    return clinical_manifest_duo

def load_process_geo(geo, annot_path="./data/docs/GPL570.annot"):
    """
    Function use:
    Load and preprocess GEO gene expression data to standard form with relevant genes (OGT/OGA).

    Inputs:
    - geo (str): GEO dataset ID (e.g., "GSEXXXXX").
    - annot_path (str): Path to the annotation file (GPL platform file).

    Outputs:
    - DataFrame: Cleaned DataFrame with sample IDs, sample types, and OGT/OGA expression.
    """
    clean_path = f"./data/GEO_GeneExpression/{geo}_relevant.csv"
    raw_path = f"./data/GEO_GeneExpression/{geo}.txt"

    if os.path.exists(clean_path):
        print("Clean file found. Loading...")
        return pd.read_csv(clean_path)

    print("Clean file not found. Processing from raw...")

    df = pd.read_csv(raw_path, sep="\t", dtype=str).dropna(how="all")
    df.iloc[:, 0:3] = df.iloc[:, 0:3].ffill()
    df = df[df["ID_REF"].notna()]
    expression_cols = df.columns[5:]
    df[expression_cols] = df[expression_cols].apply(pd.to_numeric, errors='coerce')

    melted = df.melt(
        id_vars=["!Sample_title", "!Sample_geo_accession", "!Sample_source_name_ch1", "ID_REF"],
        var_name="Sample", value_name="Expression"
    )

    pivoted = melted.pivot_table(index="Sample", columns="ID_REF", values="Expression")

    meta = df[["!Sample_title", "!Sample_geo_accession", "!Sample_source_name_ch1"]].drop_duplicates()
    meta = meta.rename(columns={
        "!Sample_geo_accession": "Sample",
        "!Sample_title": "Sample_title",
        "!Sample_source_name_ch1": "Sample_source_name"
    })

    final_df = meta.merge(pivoted, on="Sample", how="inner")

    annot = pd.read_csv(annot_path, sep="\t", comment='#', header=0, dtype=str, skiprows=27)
    valid_probes = [c for c in final_df.columns if c in annot["ID"].values]
    annot = annot[annot["ID"].isin(valid_probes)]

    probe_to_gene = annot[["ID", "Gene symbol"]].dropna()
    probe_to_gene = probe_to_gene[probe_to_gene["Gene symbol"] != ""]
    probe_to_gene = probe_to_gene.groupby("Gene symbol")["ID"].apply(list).to_dict()

    ogt_probes = probe_to_gene.get("OGT", [])

    gene_expr = {}
    for gene, probes in probe_to_gene.items():
        matching_cols = [p for p in probes if p in final_df.columns]
        if matching_cols:
            gene_expr[gene] = final_df[matching_cols].mean(axis=1)

    gene_expr_df = pd.DataFrame(gene_expr)
    gene_expr_df.index = final_df["Sample"]
    gene_expr_df = gene_expr_df.astype(float)
    gene_expr_z_df = gene_expr_df.apply(zscore, axis=0)

    ogt_expr = gene_expr_z_df.get("OGT")
    oga_expr = gene_expr_z_df.get("MGEA5")  # OGA gene 

    df_last = final_df[["Sample", "Sample_source_name"]].copy()
    df_last["OGT"] = ogt_expr.values if ogt_expr is not None else float("nan")
    df_last["OGA"] = oga_expr.values if oga_expr is not None else float("nan")
    df_last = df_last.drop_duplicates(subset=["Sample"], keep="first")

    df_last.to_csv(clean_path, index=False)
    print(f"Clean file generated and saved as {clean_path}.")
    return df_last

def environment_info():
    """
    Display Python version, platform details, and versions of all imported packages.
    """
    # Python and system information
    print("Python version:", sys.version)
    print("Platform:", platform.platform())

    # Imported package versions
    print("\nPackage Versions:")
    imported_modules = {name: module.__version__ for name, module in sys.modules.items() 
                        if hasattr(module, '__version__')}
    for package, version in sorted(imported_modules.items()):
        print(f"{package}: {version}")


In [2]:
# Display environment detail
environment_info()


Python version: 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 21:00:12) [Clang 16.0.6 ]
Platform: macOS-14.5-x86_64-i386-64bit

Package Versions:
IPython: 8.25.0
IPython.core.release: 8.25.0
PIL: 10.3.0
PIL.Image: 10.3.0
PIL._deprecate: 10.3.0
PIL._version: 10.3.0
_brotli: 1.0.9
_csv: 1.0
_ctypes: 1.1.0
_curses: b'2.2'
_decimal: 1.70
_pydev_bundle.fsnotify: 0.1.5
_pydevd_frame_eval.vendored.bytecode: 0.13.0.dev
appnope: 0.1.4
argparse: 1.1
brotli: 1.0.9
certifi: 2024.12.14
cffi: 1.16.0
charset_normalizer: 2.0.4
charset_normalizer.version: 2.0.4
comm: 0.2.2
csv: 1.0
ctypes: 1.1.0
ctypes.macholib: 1.0
cycler: 0.12.1
dateutil: 2.9.0
dateutil._version: 2.9.0
debugpy: 1.6.7
debugpy.public_api: 1.6.7
decimal: 1.70
decorator: 5.1.1
defusedxml: 0.7.1
executing: 2.0.1
executing.version: 2.0.1
http.server: 0.6
idna: 3.4
idna.idnadata: 15.0.0
idna.package_data: 3.4
ipaddress: 1.0
ipykernel: 6.29.3
ipykernel._version: 6.29.3
jedi: 0.19.1
json: 2.0.9
jupyter_client: 8.6.2
jupyter_client._v

# TCGA Gene Expression Loading

##### This script uses the GDC API to organize transcriptomic and clinical data based on manifest files (`gdc_manifest.YYYY-MM-DD.txt`) from the [GDC Data Portal](https://portal.gdc.cancer.gov/).

---

### Setup Instructions

1. **Select your cancer cohort**  
   Cohorts analyzed in the manuscript: `uterus`, `thyroid`, `lung`, `kidney`, `breast`, `bone_marrow_and_blood`

2. **Select cohort in the GDC Portal and Under `Repository` Select:**  
   - Experimental Strategy: `RNA-Seq`  
   - Data Category: `Transcriptome Profiling`  
   - Data Type: `Gene Expression Quantification`

3. **Download required files**  
   - First, click **"Download Manifest"** to obtain the `.txt` manifest.  
   - Then, use the **GDC Data Transfer Tool** to download the sample `.tsv` expression files.  
     *(Instructions for the Transfer Tool are available on the portal)*

---

### Directory Structure

Before running the script, ensure the following directory structure exists:

- `./data/TCGA_GeneExpression/{cancer}/`  
  &nbsp;&nbsp;&nbsp;&nbsp;• `gdc_manifest.YYYY-MM-DD.txt`  
  &nbsp;&nbsp;&nbsp;&nbsp;• `gene_expression/`  
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• `file1.tsv`  
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• `file2.tsv`  
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• `...`

- Replace `{cancer}` with your chosen cohort name (e.g., `uterus`).
- Place the manifest and downloaded `.tsv` files accordingly.

---

### Notes
- This setup is used to simplify data management and avoid GitHub file size limits.
- The script will read the manifest and organize local files for downstream analysis.
- To reproduce the full analysis, repeat the setup individually for each of the six cancer types by updating the `cancer` variable:



In [7]:
# Specify the cancer type to download transcriptomic and clinical data
# Analyzed in manuscript ("bone_marrow_and_blood", "lung", "kidney", "breast", "thyroid", "uterus")
cancer = "uterus"

# --- Step 1: Load the manifest file from GDC ---
# This step loads the manifest file listing metadata about available files for the specified cancer type.
gdc_manifest = load_manifest_file(f'./data/TCGA_GeneExpression/{cancer}/')

# --- Step 2: Fetch clinical data using the GDC API ---
# This step queries the GDC API to fetch clinical data such as diagnosis details and sample types.
# Ensure an active internet connection to access the GDC API.
clinical_manifest = fetch_clinical_data_for_manifest(gdc_manifest, n=None, t=0)

# Save the clinical data for reuse. This file will be saved as 'clinical_gdc_manifest.txt'.
save_clinical_manifest(clinical_manifest, f'./data/TCGA_GeneExpression/{cancer}/')

# --- Step 3: Load and clean the clinical manifest ---
# Load the saved clinical manifest file and clean it by removing irrelevant or empty columns.
clinical_manifest = load_clinical_manifest(f'./data/TCGA_GeneExpression/{cancer}/')
clinical_manifest = clean_dataframe(clinical_manifest)

# --- Step 4: Organize and compile gene expression data ---
# Ensure that the subdirectory './data/TCGA_GeneExpression/{cancer}/gene_expression/' exists
# and contains the downloaded gene expression `.tsv` files from the GDC Data Portal.

# Move all `.tsv` files from subdirectories into the 'gene_expression' directory.
organize_tsv_files(f'./data/TCGA_GeneExpression/{cancer}/gene_expression')

# Load the gene expression data from the `.tsv` files, extracting the specified feature ('fpkm_unstranded').
gene_exps, gene_key = load_gene_expression(
    clinical_manifest, f'./data/TCGA_GeneExpression/{cancer}/gene_expression', 'fpkm_unstranded'
)

# Save the processed gene expression and gene key data for later use.
save_dataframes(gene_exps, gene_key, cancer)

# --- Step 5: Analyze and verify filenames ---
# Compare filenames in the clinical manifest with those in the 'gene_expression' directory
# to identify any mismatches or missing files.
print(analyze_filenames(clinical_manifest, f'./data/TCGA_GeneExpression/{cancer}/gene_expression'))

# --- Step 6: Reload saved gene expression and gene key data ---
# Reload the saved gene expression and gene key data from the CSV files.
# This is useful for reanalysis or downstream processing without repeating earlier steps.
gene_exps, gene_key = load_dataframes(
    f'./data/TCGA_GeneExpression/{cancer}/gene_expression.csv', 
    f'./data/TCGA_GeneExpression/{cancer}/gene_key.csv'
)


100%|███████████████████████████████████████| 1168/1168 [19:29<00:00,  1.00s/it]


File saved successfully at ./data/TCGA_GeneExpression/uterus/clinical_gdc_manifest.txt
Clinical manifest shape: (1168, 103)
Cleaned clinical manifest shape: (1168, 48)
No directories to clean. Cleaning done.
Gene expression shape: (1168, 60660)
Gene key shape: (60660, 2)
{'percent_unique_filenames_in_manifest': 100.0, 'percent_overlap_with_directory': 100.0, 'percent_non_overlap_with_directory': 0.0}


In [8]:
gene_key.head()

,gene_id,gene_name
0,ENSG00000000003.15,TSPAN6
1,ENSG00000000005.6,TNMD
2,ENSG00000000419.13,DPM1
3,ENSG00000000457.14,SCYL3
4,ENSG00000000460.17,C1orf112


In [9]:
gene_exps.head()

,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
filename,,,,,,,,,,,,,,,,,,,,,
6a6ccdf5-9405-409a-a161-72e875400889.rna_seq.augmented_star_gene_counts.tsv,19.2975,1.4941,12.5086,6.7530,3.9497,1.4546,16.7088,12.2630,20.4519,16.5588,...,0.0,36.7550,0.8335,0.0,44.4953,0.0,8.2613,0.0,0.0176,1.0098
e5325dd8-867a-4cc2-8b93-9b0b58fc32f3.rna_seq.augmented_star_gene_counts.tsv,10.6894,0.3790,35.1229,5.4748,2.5248,3.5777,3.1174,8.7109,4.1676,13.2483,...,0.0,7.3599,0.1395,0.0,11.5829,0.0,5.5882,0.0,0.0673,0.0350
75073929-f7b1-4165-a786-60f733b962d9.rna_seq.augmented_star_gene_counts.tsv,17.7992,0.4563,26.4298,5.0877,2.7658,4.6093,29.4178,15.9268,8.3757,9.7255,...,0.0,6.6512,0.1094,0.0,13.9021,0.0,2.9262,0.0,0.0389,0.3266
119ac668-eab5-42e5-a7a5-f28720173235.rna_seq.augmented_star_gene_counts.tsv,10.4508,0.3030,18.1245,9.1775,5.8430,0.8595,11.2738,14.0382,4.9385,15.3237,...,0.0,9.9381,0.1635,0.0,25.6598,0.0,5.1973,0.0,0.0387,0.2218
09c2ef25-0496-4574-995b-d493f9ddf724.rna_seq.augmented_star_gene_counts.tsv,27.1184,0.3620,26.7232,6.2870,3.4341,3.0606,55.9515,17.7718,20.5075,15.8846,...,0.0,14.4691,0.0762,0.0,20.8004,0.0,3.2337,0.0,0.0347,0.1391


In [52]:
tcga_bmb = load_process_tcga("bone_marrow_and_blood")
tcga_lng = load_process_tcga("lung")
tcga_kid = load_process_tcga("kidney")
tcga_brs = load_process_tcga("breast")
tcga_thy = load_process_tcga("thyroid")
tcga_utr = load_process_tcga("uterus")

[proccessing] data for: bone_marrow_and_blood
Clinical manifest shape: (7224, 102)
Cleaned data saved to: ./data/TCGA_GeneExpression/bone_marrow_and_blood/clean_expression_data.csv

[proccessing] data for: lung
Clinical manifest shape: (1996, 102)
Cleaned data saved to: ./data/TCGA_GeneExpression/lung/clean_expression_data.csv

[proccessing] data for: kidney
Clinical manifest shape: (1842, 102)
Cleaned data saved to: ./data/TCGA_GeneExpression/kidney/clean_expression_data.csv

[proccessing] data for: breast
Clinical manifest shape: (1594, 101)
Cleaned data saved to: ./data/TCGA_GeneExpression/breast/clean_expression_data.csv

[proccessing] data for: thyroid
Clinical manifest shape: (1402, 177)
Cleaned data saved to: ./data/TCGA_GeneExpression/thyroid/clean_expression_data.csv

[proccessing] data for: uterus
Clinical manifest shape: (1168, 103)
Cleaned data saved to: ./data/TCGA_GeneExpression/uterus/clean_expression_data.csv



In [56]:
tcga_utr.head()

,filename,OGT,OGA,label,cancer_stage
0,6a6ccdf5-9405-409a-a161-72e875400889.rna_seq.a...,0.975632,0.769566,Cancer,NaN
1,e5325dd8-867a-4cc2-8b93-9b0b58fc32f3.rna_seq.a...,0.176287,0.167766,Healthy,Healthy
2,75073929-f7b1-4165-a786-60f733b962d9.rna_seq.a...,0.112565,0.104459,Cancer,NaN
3,119ac668-eab5-42e5-a7a5-f28720173235.rna_seq.a...,0.533852,0.318600,Cancer,NaN
4,09c2ef25-0496-4574-995b-d493f9ddf724.rna_seq.a...,0.183527,0.099547,Cancer,NaN


# External GEO Gene Expression Loading

##### This script loads and processes gene expression data from GEO (series matrix `.txt` files) to extract OGT and OGA expression along with basic clinical metadata (cancer status).

---

### Setup Instructions

1. **GEO Datasets Used**  
   Each GEO dataset corresponds to a cancer type in the manuscript:

   - `GSE13159` → **Bone Marrow & Blood**
   - `GSE19804` → **Lung**
   - `GSE66271` + `GSE53757` → **Kidney**
   - `GSE42568` → **Breast**
   - `GSE58545` → **Thyroid**
   - `GSE17025` → **Uterus**
     

2. **Download Required Files**  
   - Go to the [GEO DataSets Portal](https://www.ncbi.nlm.nih.gov/geo/)
   - Search for each GSE ID and download the **Series Matrix File(s)** (`.txt`)
   - Save each file to the directory below

3. **Input File Format Requirement**  
   - First 3 columns: `!Sample_title`, `!Sample_geo_accession`, `!Sample_source_name_ch1`  
   - 4th column: (empty column separating metadata from data)  
   - Remaining: probe expression matrix (rows = probes, columns = samples)

---

### Directory Structure

Place downloaded input and annotation files in:

**GEO Input Files**  
- `./data/GEO_GeneExpression/`  
  &nbsp;&nbsp;&nbsp;&nbsp;• `GSE13159.txt`  
  &nbsp;&nbsp;&nbsp;&nbsp;• `GSE19804.txt`  
  &nbsp;&nbsp;&nbsp;&nbsp;• `...`  


**Platform Annotation File**  
- `./data/docs/`  
  &nbsp;&nbsp;&nbsp;&nbsp;• `GPL570.annot`

---

### Notes

- GEO files must be downloaded manually and follow the required format  
- Cleaned `.csv` versions are provided in the repository; raw `.txt` files are too large for GitHub  
- Only OGT and OGA are extracted 



In [3]:
# If no clean file exists - this will generate GSEXXXXX_relevant.csv
# Using the downloaded series matrix files
geo_bmb = load_process_geo(geo= 'GSE13159')
geo_lng = load_process_geo(geo= 'GSE19804')
geo_kid = load_process_geo(geo= 'GSE66271') # GSE53757
geo_brs = load_process_geo(geo= 'GSE42568')
geo_thy = load_process_geo(geo= 'GSE58545')
geo_utr = load_process_geo(geo= 'GSE17025')


Clean file not found. Processing from raw...
Clean file generated and saved as ./data/GEO_GeneExpression/GSE19804_relevant.csv.
Clean file not found. Processing from raw...
Clean file generated and saved as ./data/GEO_GeneExpression/GSE66271_relevant.csv.
Clean file not found. Processing from raw...
Clean file generated and saved as ./data/GEO_GeneExpression/GSE53757_relevant.csv.
Clean file not found. Processing from raw...
Clean file generated and saved as ./data/GEO_GeneExpression/GSE42568_relevant.csv.
Clean file not found. Processing from raw...
Clean file generated and saved as ./data/GEO_GeneExpression/GSE58545_relevant.csv.
Clean file not found. Processing from raw...
Clean file generated and saved as ./data/GEO_GeneExpression/GSE17025_relevant.csv.


In [4]:
geo_utr.head()

,Sample,Sample_source_name,OGT,OGA
0,GSM425837,stage: IC,-0.542185,-0.253381
1,GSM425838,stage: IC,2.262353,4.286525
2,GSM425839,stage: IC,1.035677,1.050511
3,GSM425840,stage: IC,-0.201921,-0.663616
4,GSM425841,stage: IC,-0.664804,-0.003481


In [5]:
# If clean GSEXXXXX_relevant.csv file exists - this will load it
geo_utr = load_process_geo(geo= 'GSE17025')
geo_utr.head()

Clean file found. Loading...


,Sample,Sample_source_name,OGT,OGA
0,GSM425837,stage: IC,-0.542185,-0.253381
1,GSM425838,stage: IC,2.262353,4.286525
2,GSM425839,stage: IC,1.035677,1.050511
3,GSM425840,stage: IC,-0.201921,-0.663616
4,GSM425841,stage: IC,-0.664804,-0.003481
